# Coutning of maximal independent sets in the hypercube covergraph with a modified version of CloseByOne 

## Part 1. Counting maximal idependent sets


In [2]:
%load_ext Cython

In [2]:
%%cython --annotate
#import copy

import myglobals

from importlib import reload  


def lst2int(lst):
    r=0
    for i in lst:
        r |= (1<<i)
    return r 

def lst2int_(lst,cxt):
    r=0
    for i in lst:
        r |= (1<<(cxt.nObj-i-1))
    return r 

def int2lst(n):
    ret=[]
    sft=0
    if n==0:
        return ret
    while n!=0:
        if n&1==1:
            ret.append(sft)
        n=n>>1
        sft+=1            
    return ret

def renameList(lst,cxt):
    ret=[]
    for i in lst[::-1]:
        ret.append(cxt.nObj-i-1)
    return ret


def int2lst_(n,cxt):
    ret=[]
    sft=0
    if n==0:
        return ret
    while n!=0:
        if n&1==1:
            ret.append(cxt.nObj-sft-1)
        n=n>>1
        sft+=1            
    return ret
    

class nnBitContext:
    def __init__(self, n=3, i=0, m=3):
            self.attr_names=[str(m+1) for m in range(n)]
            self.objects=[]
            #print(bin(i))
            for j in range(m):                
                self.objects.append(i & ((1<<n) - 1))
                #print(bin(i & ((1<<n) - 1)))
                i=i>>n
                #print(bin(i))
            self.nAttr=len(self.attr_names)
            self.nObj=len(self.objects)


      
 
    def aClosure(self,aSet):
        #compute attribute closure
        B=(1<<self.nAttr)-1
        for g in self.objects:
            if aSet & g == aSet:
                B=B & g        
        return B
   
    def oClosure(self,oSet):
        #compute object closure
        B=self.oPrime(oSet)
        A=[]
        for i in range(self.nObj):
            if B & self.objects[i] == B:
                A.append(i) 
        #print("A",A)
        return A

    def oPrime(self,oSet):
        #object prime takes a list of objects (indicies) and results in an integer (bitset)
        B=(1<<self.nAttr)-1
        for i in oSet:
            B=B&self.objects[i]
        return B
    
    def aPrime(self,aSet):
        #attribute prime takes an integer (bitset of attributes)
        #and results in a list of objects (the extent)
        A=[]
        for i in range(self.nObj):
            if aSet & self.objects[i]==aSet: 
                A.append(i)
        return A
    
def cxtFromFile(filename):
    cxt=nnBitContext(0,0,0)
   
    f = open(filename, "r")
    print(f.readline())
    print(f.readline())
    
    cxt.nAttr=int(f.readline())
    cxt.nObj=int(f.readline())
    
    print(f.readline())
    
    for i in range(cxt.nObj):
        print(f.readline())
    
    for i in range(cxt.nAttr):
        cxt.attr_names.append(f.readline())
    

    for l in range(cxt.nObj):
        o=f.readline().replace('.','0').replace('x','1')
        print(o)
        
        cxt.objects.append(int(o,2))
        #print(bin(i & ((1<<n) - 1)))
        
        #print(bin(i))
    #print(cxt.objects)
    #cxt.objects.reverse()
    #print("---reverse---")
    #print(cxt.objects)
    return cxt         

reload(myglobals)
myglobals.bCxt=cxtFromFile("CubeInd2.cxt")



def nu(n):
    ret=0
    if n==0:
        return 0
    while n!=0:
        if n&1==1:
            ret+=1
        n=n>>1
    return ret

B





0

1

2

3

1001

0110

0110

1001



In [4]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp

import myglobals

from importlib import reload 

reload(myglobals)
#myglobals.bCxt=cxtFromFile("MA7.cxt")

myglobals.bCxt=cxtFromFile("CubeInd3.cxt")







def CbOFiller(A,g,C,D):
    cnt=0
    parList=[]
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
        cnt+=1
        print("Filler:",(C,int2lst(D)))
        print("Filler:",(renameList(C,myglobals.bCxt),int2lst(D)))
        print("Filler:",lst2int(C),D)
        print("Filler:",lst2int_(C,myglobals.bCxt),D)

        

        F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
        for f in F:
            Z=C[:]
            Z.append(f)
            Y = D & myglobals.bCxt.objects[f]
            X = myglobals.bCxt.aPrime(Y)
            #print((X,int2lst(Y)))
            
            parList.append([Z,f,X,Y])
            
    return cnt,parList


    
    

def processCbO(A,g,C,D):
    
    
 
  
    cnt=0
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
        cnt+=1
        print("\t",(C,int2lst(D)))
        print("\t",(renameList(C,myglobals.bCxt),int2lst(D)))
        print("\t",lst2int(C),D)
        print("\t",lst2int_(C,myglobals.bCxt),D)
        
        if lst2int_(C,myglobals.bCxt)==D:
            print("Max clique:",lst2int_(C,myglobals.bCxt),D)
            print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))


        F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
        for f in F:
            Z=C[:]
            Z.append(f)
            Y = D & myglobals.bCxt.objects[f]
            X = myglobals.bCxt.aPrime(Y)
            
            
            cnt+=processCbO(Z,f,X,Y)
    
    if cnt>0 and cnt%10000==0: print(g,cnt)
    
    return cnt



 



if __name__ == "__main__":
    

    #pool = Pool(cpu_count())
    pool = Pool(1)
    

    
    cnt=0
    
    parList=[]
    
    if myglobals.bCxt.oClosure([])==[]:
        cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))


        parList.extend(lparList)
        
        cnt+=lcnt
    
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res8t = pool.starmap_async(processCbO, parList)
    print(len(res8t.get()))
    print("total:",sum(res8t.get())+cnt)
    from math import sqrt
    print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

B





0

1

2

3

4

5

6

7

10010111

01101011

01101101

10011110

01111001

10110110

11010110

11101001

	 ([0, 1], [0, 1])
	 ([6, 7], [0, 1])
	 3 3
	 192 3
	 ([0, 1, 2, 4, 7], [0])
	 ([0, 3, 5, 6, 7], [0])
	 151 1
	 233 1
	 ([0, 1, 2, 3, 4, 5, 6, 7], [])
	 ([0, 1, 2, 3, 4, 5, 6, 7], [])
	 255 0
	 255 0
	 ([0, 1, 3, 5, 6], [1])
	 ([1, 2, 4, 6, 7], [1])
	 107 2
	 214 2
	 ([0, 2], [0, 2])
	 ([5, 7], [0, 2])
	 5 5
	 160 5
	 ([0, 2, 3, 5, 6], [2])
	 ([1, 2, 4, 5, 7], [2])
	 109 4
	 182 4
	 ([0, 3, 5, 6], [1, 2, 4, 7])
	 ([1, 2, 4, 7], [1, 2, 4, 7])
	 105 150
	 150 150
Max clique: 150 150
Max clique: ([1, 2, 4, 7], [1, 2, 4, 7])
	 ([0, 3, 4, 5, 6], [4])
	 ([1, 2, 3, 4, 7], [4])
	 121 16
	 158 16
	 ([0, 3, 5, 6, 7], [7])
	 ([0, 1, 2, 4, 7], [7])
	 233 128
	Filler: ([0], [0, 1, 2, 4, 7])
Filler: ([7], [0, 1, 2, 4, 7])
Filler: 1 151
Filler: 128 151
Filler: ([1], [0, 1, 3, 5, 6])
Filler: ([6], [0, 1, 3, 5, 6])
Filler: 2 107
Filler: 64 107
Filler: ([2], [0, 2, 3, 5, 6])
Filler: ([5], [0, 2

In [3]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp

import myglobals

from importlib import reload 

reload(myglobals)
#myglobals.bCxt=cxtFromFile("MA7.cxt")

myglobals.bCxt=cxtFromFile("CubeInd3.cxt")







def CbOFiller(A,g,C,D):
    cnt=0
    parList=[]
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
#         cnt+=1
#         print("Filler:",(C,int2lst(D)))
#         print("Filler:",(renameList(C,myglobals.bCxt),int2lst(D)))
#         print("Filler:",lst2int(C),D)
#         print("Filler:",lst2int_(C,myglobals.bCxt),D)

        intC=lst2int_(C,myglobals.bCxt)

        if intC==D:
            cnt+=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)
                #print((X,int2lst(Y)))

                parList.append([Z,f,X,Y])

    return cnt,parList


    
    

def processCbO(A,g,C,D):
    
    
 
  
    cnt=0
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
        #cnt+=1
        #print("\t",(C,int2lst(D)))
        #print("\t",(renameList(C,myglobals.bCxt),int2lst(D)))
        #print("\t",lst2int(C),D)
        #print("\t",lst2int_(C,myglobals.bCxt),D)
        intC=lst2int_(C,myglobals.bCxt)

        
        if intC==D:
            print("Max clique:",intC,D)
            print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))
            cnt+=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)


                cnt+=processCbO(Z,f,X,Y)
    
    if cnt>0 and cnt%10000==0: print(g,cnt)
    
    return cnt



 



if __name__ == "__main__":
    

    #pool = Pool(cpu_count())
    pool = Pool(1)
    

    
    cnt=0
    
    parList=[]
    
    #if myglobals.bCxt.oClosure([])==[]:
    #    cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))


        parList.extend(lparList)
        
        cnt+=lcnt
    
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res8t = pool.starmap_async(processCbO, parList)
    print(len(res8t.get()))
    print("total:",sum(res8t.get())+cnt)
    #from math import sqrt
    #print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'v

B





0

1

2

3

4

5

6

7

10010111

01101011

01101101

10011110

01111001

10110110

11010110

11101001

Max clique: 150 150
Max clique: ([1, 2, 4, 7], [1, 2, 4, 7])
Max clique: 129 129
Max clique: ([0, 7], [0, 7])
Max clique: 105 105
Max clique: ([0, 3, 5, 6], [0, 3, 5, 6])
Max clique: 66 66
Max clique: ([1, 6], [1, 6])
Max clique: 36 36
Max clique: ([2, 5], [2, 5])
Max clique: 24 24
Max clique: ([3, 4], [3, 4])
[[[0, 1], 1, [0, 1], 3], [[0, 2], 2, [0, 2], 5], [[0, 3], 3, [0, 3, 5, 6], 150], [[0, 4], 4, [0, 4], 17], [[0, 5], 5, [0, 3, 5, 6], 150], [[0, 6], 6, [0, 3, 5, 6], 150], [[0, 7], 7, [0, 7], 129], [[1, 2], 2, [1, 2, 4, 7], 105], [[1, 3], 3, [1, 3], 10], [[1, 4], 4, [1, 2, 4, 7], 105], [[1, 5], 5, [1, 5], 34], [[1, 6], 6, [1, 6], 66], [[1, 7], 7, [1, 2, 4, 7], 105], [[2, 3], 3, [2, 3], 12], [[2, 4], 4, [1, 2, 4, 7], 105], [[2, 5], 5, [2, 5], 36], [[2, 6], 6, [2, 6], 68], [[2, 7], 7, [1, 2, 4, 7], 105], [[3, 4], 4, [3, 4], 24], [[3, 5], 5, [0, 3, 5, 6], 150], [[3, 6], 6, [0

In [4]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp

import myglobals

from importlib import reload 

reload(myglobals)
#myglobals.bCxt=cxtFromFile("MA7.cxt")

myglobals.bCxt=cxtFromFile("CubeInd4.cxt")







def CbOFiller(A,g,C,D):
    cnt=0
    parList=[]
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
#         cnt+=1
#         print("Filler:",(C,int2lst(D)))
#         print("Filler:",(renameList(C,myglobals.bCxt),int2lst(D)))
#         print("Filler:",lst2int(C),D)
#         print("Filler:",lst2int_(C,myglobals.bCxt),D)

        intC=lst2int_(C,myglobals.bCxt)

        if intC==D:
            cnt+=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)
                #print((X,int2lst(Y)))

                parList.append([Z,f,X,Y])

    return cnt,parList


    
    

def processCbO(A,g,C,D):
    
    
 
  
    cnt=0
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
        #cnt+=1
        #print("\t",(C,int2lst(D)))
        #print("\t",(renameList(C,myglobals.bCxt),int2lst(D)))
        #print("\t",lst2int(C),D)
        #print("\t",lst2int_(C,myglobals.bCxt),D)
        intC=lst2int_(C,myglobals.bCxt)

        
        if intC==D:
            print("Max clique:",intC,D)
            print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))
            cnt+=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)


                cnt+=processCbO(Z,f,X,Y)
    
    if cnt>0 and cnt%10000==0: print(g,cnt)
    
    return cnt



 



if __name__ == "__main__":
    

    #pool = Pool(cpu_count())
    pool = Pool(1)
    

    
    cnt=0
    
    parList=[]
    
    #if myglobals.bCxt.oClosure([])==[]:
    #    cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))


        parList.extend(lparList)
        
        cnt+=lcnt
    
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res8t = pool.starmap_async(processCbO, parList)
    print(len(res8t.get()))
    print("total:",sum(res8t.get())+cnt)
    #from math import sqrt
    #print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'v

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

1001011101111111

0110101110111111

0110110111011111

1001111011101111

0111100111110111

1011011011111011

1101011011111101

1110100111111110

0111111110010111

1011111101101011

1101111101101101

1110111110011110

1111011101111001

1111101110110110

1111110111010110

1111111011101001

Max clique: 38505 38505
Max clique: ([0, 3, 5, 6, 9, 10, 12, 15], [0, 3, 5, 6, 9, 10, 12, 15])
Max clique: 37954 37954
Max clique: ([1, 6, 10, 12, 15], [1, 6, 10, 12, 15])
Max clique: 37412 37412
Max clique: ([2, 5, 9, 12, 15], [2, 5, 9, 12, 15])
Max clique: 36870 36870
Max clique: ([1, 2, 12, 15], [1, 2, 12, 15])
Max clique: 34328 34328
Max clique: ([3, 4, 9, 10, 15], [3, 4, 9, 10, 15])
Max clique: 33810 33810
Max clique: ([1, 4, 10, 15], [1, 4, 10, 15])
Max clique: 33300 33300
Max clique: ([2, 4, 9, 15], [2, 4, 9, 15])
Max clique: 33128 33128
Max clique: ([3, 5, 6, 8, 15], [3, 5, 6, 8, 15])
Max clique: 33090 33090
Max clique: ([1, 6, 8, 15],

In [5]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp

import myglobals

from importlib import reload 

reload(myglobals)
#myglobals.bCxt=cxtFromFile("MA7.cxt")

myglobals.bCxt=cxtFromFile("CubeInd5.cxt")







def CbOFiller(A,g,C,D):
    cnt=0
    parList=[]
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
#         cnt+=1
#         print("Filler:",(C,int2lst(D)))
#         print("Filler:",(renameList(C,myglobals.bCxt),int2lst(D)))
#         print("Filler:",lst2int(C),D)
#         print("Filler:",lst2int_(C,myglobals.bCxt),D)

        intC=lst2int_(C,myglobals.bCxt)

        if intC==D:
            cnt+=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)
                #print((X,int2lst(Y)))

                parList.append([Z,f,X,Y])

    return cnt,parList


    
    

def processCbO(A,g,C,D):
    
    
 
  
    cnt=0
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
        #cnt+=1
        #print("\t",(C,int2lst(D)))
        #print("\t",(renameList(C,myglobals.bCxt),int2lst(D)))
        #print("\t",lst2int(C),D)
        #print("\t",lst2int_(C,myglobals.bCxt),D)
        intC=lst2int_(C,myglobals.bCxt)

        
        if intC==D:
            print("Max clique:",intC,D)
            print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))
            cnt+=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)


                cnt+=processCbO(Z,f,X,Y)
    
    if cnt>0 and cnt%10000==0: print(g,cnt)
    
    return cnt



 



if __name__ == "__main__":
    

    #pool = Pool(cpu_count())
    pool = Pool(1)
    

    
    cnt=0
    
    parList=[]
    
    #if myglobals.bCxt.oClosure([])==[]:
    #    cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))


        parList.extend(lparList)
        
        cnt+=lcnt
    
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res8t = pool.starmap_async(processCbO, parList)
    print(len(res8t.get()))
    print("total:",sum(res8t.get())+cnt)
    #from math import sqrt
    #print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'v

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

10010111011111110111111111111111

01101011101111111011111111111111

01101101110111111101111111111111

10011110111011111110111111111111

01111001111101111111011111111111

10110110111110111111101111111111

11010110111111011111110111111111

11101001111111101111111011111111

01111111100101111111111101111111

10111111011010111111111110111111

11011111011011011111111111011111

11101111100111101111111111101111

11110111011110011111111111110111

11111011101101101111111111111011

11111101110101101111111111111101

11111110111010011111111111111110

01111111111111111001011101111111

10111111111111110110101110111111

11011111111111110110110111011111

11101111111111111001111011101111

11110111111111110111100111110111

11111011111111111011011011111011

11111101111111111101011011111101

11111110111111111110100111111110

11111111011111110111111110010111

1111111110111111101111110

Max clique: ([0, 3, 5, 6, 25, 26, 28, 31], [0, 3, 5, 6, 25, 26, 28, 31])
Max clique: 2489926292 2489926292
Max clique: ([2, 4, 7, 9, 14, 16, 19, 21, 22, 26, 28, 31], [2, 4, 7, 9, 14, 16, 19, 21, 22, 26, 28, 31])
Max clique: 2489860737 2489860737
Max clique: ([0, 7, 9, 14, 19, 21, 22, 26, 28, 31], [0, 7, 9, 14, 19, 21, 22, 26, 28, 31])
Max clique: 2489401880 2489401880
Max clique: ([3, 4, 9, 14, 16, 21, 22, 26, 28, 31], [3, 4, 9, 14, 16, 21, 22, 26, 28, 31])
Max clique: 2489336329 2489336329
Max clique: ([0, 3, 9, 14, 21, 22, 26, 28, 31], [0, 3, 9, 14, 21, 22, 26, 28, 31])
Max clique: 2487829028 2487829028
Max clique: ([2, 5, 9, 14, 16, 19, 22, 26, 28, 31], [2, 5, 9, 14, 16, 19, 22, 26, 28, 31])
Max clique: 2487763489 2487763489
Max clique: ([0, 5, 9, 14, 19, 22, 26, 28, 31], [0, 5, 9, 14, 19, 22, 26, 28, 31])
Max clique: 2487380372 2487380372
Max clique: ([2, 4, 7, 8, 11, 13, 14, 17, 22, 26, 28, 31], [2, 4, 7, 8, 11, 13, 14, 17, 22, 26, 28, 31])
Max clique: 2487380097 2487380097
Max cl

Max clique: 2249726081 2249726081
Max clique: ([0, 7, 11, 12, 19, 20, 25, 26, 31], [0, 7, 11, 12, 19, 20, 25, 26, 31])
Max clique: 2249726049 2249726049
Max clique: ([0, 5, 6, 11, 12, 19, 20, 25, 26, 31], [0, 5, 6, 11, 12, 19, 20, 25, 26, 31])
Max clique: 2249726018 2249726018
Max clique: ([1, 6, 11, 12, 19, 20, 25, 26, 31], [1, 6, 11, 12, 19, 20, 25, 26, 31])
Max clique: 2249725988 2249725988
Max clique: ([2, 5, 11, 12, 19, 20, 25, 26, 31], [2, 5, 11, 12, 19, 20, 25, 26, 31])
Max clique: 2249722208 2249722208
Max clique: ([5, 6, 8, 11, 19, 20, 25, 26, 31], [5, 6, 8, 11, 19, 20, 25, 26, 31])
Max clique: 2249220360 2249220360
Max clique: ([3, 8, 13, 14, 20, 25, 26, 31], [3, 8, 13, 14, 20, 25, 26, 31])
Max clique: 2249220105 2249220105
Max clique: ([0, 3, 13, 14, 20, 25, 26, 31], [0, 3, 13, 14, 20, 25, 26, 31])
Max clique: 2249212200 2249212200
Max clique: ([3, 5, 8, 14, 20, 25, 26, 31], [3, 5, 8, 14, 20, 25, 26, 31])
Max clique: 2249211945 2249211945
Max clique: ([0, 3, 5, 14, 20, 25, 2

Max clique: 2168996388 2168996388
Max clique: ([2, 5, 9, 14, 19, 22, 24, 31], [2, 5, 9, 14, 19, 22, 24, 31])
Max clique: 2168996385 2168996385
Max clique: ([0, 5, 9, 14, 19, 22, 24, 31], [0, 5, 9, 14, 19, 22, 24, 31])
Max clique: 2168985636 2168985636
Max clique: ([2, 5, 11, 12, 19, 22, 24, 31], [2, 5, 11, 12, 19, 22, 24, 31])
Max clique: 2168985633 2168985633
Max clique: ([0, 5, 11, 12, 19, 22, 24, 31], [0, 5, 11, 12, 19, 22, 24, 31])
Max clique: 2168985121 2168985121
Max clique: ([0, 5, 9, 10, 12, 19, 22, 24, 31], [0, 5, 9, 10, 12, 19, 22, 24, 31])
Max clique: 2168984100 2168984100
Max clique: ([2, 5, 9, 12, 19, 22, 24, 31], [2, 5, 9, 12, 19, 22, 24, 31])
Max clique: 2168613012 2168613012
Max clique: ([2, 4, 7, 11, 13, 14, 17, 22, 24, 31], [2, 4, 7, 11, 13, 14, 17, 22, 24, 31])
Max clique: 2168612993 2168612993
Max clique: ([0, 7, 11, 13, 14, 17, 22, 24, 31], [0, 7, 11, 13, 14, 17, 22, 24, 31])
Max clique: 2168610840 2168610840
Max clique: ([3, 4, 13, 14, 17, 22, 24, 31], [3, 4, 13, 

Max clique: ([2, 5, 9, 12, 15, 17, 20, 23, 24, 27, 29, 30], [2, 5, 9, 12, 15, 17, 20, 23, 24, 27, 29, 30])
Max clique: 1771081734 1771081734
Max clique: ([1, 2, 12, 15, 20, 23, 24, 27, 29, 30], [1, 2, 12, 15, 20, 23, 24, 27, 29, 30])
Max clique: 1770423832 1770423832
Max clique: ([3, 4, 9, 10, 15, 17, 18, 23, 24, 27, 29, 30], [3, 4, 9, 10, 15, 17, 18, 23, 24, 27, 29, 30])
Max clique: 1770292242 1770292242
Max clique: ([1, 4, 10, 15, 18, 23, 24, 27, 29, 30], [1, 4, 10, 15, 18, 23, 24, 27, 29, 30])
Max clique: 1770160660 1770160660
Max clique: ([2, 4, 9, 15, 17, 23, 24, 27, 29, 30], [2, 4, 9, 15, 17, 23, 24, 27, 29, 30])
Max clique: 1770029078 1770029078
Max clique: ([1, 2, 4, 15, 23, 24, 27, 29, 30], [1, 2, 4, 15, 23, 24, 27, 29, 30])
Max clique: 1763055233 1763055233
Max clique: ([0, 7, 9, 10, 12, 17, 18, 20, 24, 27, 29, 30], [0, 7, 9, 10, 12, 17, 18, 20, 24, 27, 29, 30])
Max clique: 1762923650 1762923650
Max clique: ([1, 7, 10, 12, 18, 20, 24, 27, 29, 30], [1, 7, 10, 12, 18, 20, 24, 2

...

Max clique: ([1, 2, 7, 11, 12, 16, 19, 22, 26, 29], [1, 2, 7, 11, 12, 16, 19, 22, 26, 29])
Max clique: 608770084 608770084
Max clique: ([2, 5, 11, 12, 16, 19, 22, 26, 29], [2, 5, 11, 12, 16, 19, 22, 26, 29])
Max clique: 608768644 608768644
Max clique: ([2, 7, 9, 12, 16, 19, 22, 26, 29], [2, 7, 9, 12, 16, 19, 22, 26, 29])
Max clique: 608735777 608735777
Max clique: ([0, 5, 9, 12, 15, 19, 22, 26, 29], [0, 5, 9, 12, 15, 19, 22, 26, 29])
Max clique: 608716929 608716929
Max clique: ([0, 7, 11, 14, 19, 22, 26, 29], [0, 7, 11, 14, 19, 22, 26, 29])
Max clique: 608716833 608716833
Max clique: ([0, 5, 11, 14, 19, 22, 26, 29], [0, 5, 11, 14, 19, 22, 26, 29])
Max clique: 608715393 608715393
Max clique: ([0, 7, 9, 14, 19, 22, 26, 29], [0, 7, 9, 14, 19, 22, 26, 29])
Max clique: 608715297 608715297
Max clique: ([0, 5, 9, 14, 19, 22, 26, 29], [0, 5, 9, 14, 19, 22, 26, 29])
Max clique: 608704641 608704641
Max clique: ([0, 7, 11, 12, 19, 22, 26, 29], [0, 7, 11, 12, 19, 22, 26, 29])
Max clique: 608704545

Max clique: ([1, 7, 10, 12, 16, 19, 22, 29], [1, 7, 10, 12, 16, 19, 22, 29])
Max clique: 541656720 541656720
Max clique: ([4, 7, 9, 10, 16, 19, 22, 29], [4, 7, 9, 10, 16, 19, 22, 29])
Max clique: 541656210 541656210
Max clique: ([1, 4, 7, 10, 16, 19, 22, 29], [1, 4, 7, 10, 16, 19, 22, 29])
Max clique: 538331520 538331520
Max clique: ([7, 8, 11, 14, 17, 18, 20, 29], [7, 8, 11, 14, 17, 18, 20, 29])
Max clique: 538200450 538200450
Max clique: ([1, 7, 8, 11, 14, 18, 20, 29], [1, 7, 8, 11, 14, 18, 20, 29])
Max clique: 537282960 537282960
Max clique: ([4, 7, 8, 11, 14, 17, 18, 29], [4, 7, 8, 11, 14, 17, 18, 29])
Max clique: 537151890 537151890
Max clique: ([1, 4, 7, 8, 11, 14, 18, 29], [1, 4, 7, 8, 11, 14, 18, 29])
Max clique: 411469417 411469417
Max clique: ([0, 3, 5, 6, 9, 10, 15, 17, 18, 23, 27, 28], [0, 3, 5, 6, 9, 10, 15, 17, 18, 23, 27, 28])
Max clique: 411469336 411469336
Max clique: ([3, 4, 9, 10, 15, 17, 18, 23, 27, 28], [3, 4, 9, 10, 15, 17, 18, 23, 27, 28])
Max clique: 411468136 4

Max clique: 408969345 408969345
Max clique: ([0, 7, 13, 14, 21, 22, 27, 28], [0, 7, 13, 14, 21, 22, 27, 28])
Max clique: 408969225 408969225
Max clique: ([0, 3, 13, 14, 21, 22, 27, 28], [0, 3, 13, 14, 21, 22, 27, 28])
Max clique: 408961665 408961665
Max clique: ([0, 7, 9, 14, 21, 22, 27, 28], [0, 7, 9, 14, 21, 22, 27, 28])
Max clique: 408961545 408961545
Max clique: ([0, 3, 9, 14, 21, 22, 27, 28], [0, 3, 9, 14, 21, 22, 27, 28])
Max clique: 408953985 408953985
Max clique: ([0, 7, 10, 13, 21, 22, 27, 28], [0, 7, 10, 13, 21, 22, 27, 28])
Max clique: 408953865 408953865
Max clique: ([0, 3, 10, 13, 21, 22, 27, 28], [0, 3, 10, 13, 21, 22, 27, 28])
Max clique: 408946305 408946305
Max clique: ([0, 7, 9, 10, 21, 22, 27, 28], [0, 7, 9, 10, 21, 22, 27, 28])
Max clique: 407012905 407012905
Max clique: ([0, 3, 5, 9, 10, 15, 17, 22, 27, 28], [0, 3, 5, 9, 10, 15, 17, 22, 27, 28])
Max clique: 407012888 407012888
Max clique: ([3, 4, 9, 10, 15, 17, 22, 27, 28], [3, 4, 9, 10, 15, 17, 22, 27, 28])
Max cli

Max clique: ([2, 4, 9, 14, 16, 23, 26, 28], [2, 4, 9, 14, 16, 23, 26, 28])
Max clique: 342458900 342458900
Max clique: ([2, 4, 9, 15, 16, 19, 21, 22, 26, 28], [2, 4, 9, 15, 16, 19, 21, 22, 26, 28])
Max clique: 342393345 342393345
Max clique: ([0, 9, 15, 19, 21, 22, 26, 28], [0, 9, 15, 19, 21, 22, 26, 28])
Max clique: 341934616 341934616
Max clique: ([3, 4, 9, 15, 16, 21, 22, 26, 28], [3, 4, 9, 15, 16, 21, 22, 26, 28])
Max clique: 341869065 341869065
Max clique: ([0, 3, 9, 15, 21, 22, 26, 28], [0, 3, 9, 15, 21, 22, 26, 28])
Max clique: 340361764 340361764
Max clique: ([2, 5, 9, 15, 16, 19, 22, 26, 28], [2, 5, 9, 15, 16, 19, 22, 26, 28])
Max clique: 340296225 340296225
Max clique: ([0, 5, 9, 15, 19, 22, 26, 28], [0, 5, 9, 15, 19, 22, 26, 28])
Max clique: 339903017 339903017
Max clique: ([0, 3, 5, 9, 15, 17, 22, 26, 28], [0, 3, 5, 9, 15, 17, 22, 26, 28])
Max clique: 339903012 339903012
Max clique: ([2, 5, 9, 15, 17, 22, 26, 28], [2, 5, 9, 15, 17, 22, 26, 28])
Max clique: 339903000 3399030

Max clique: ([2, 8, 13, 14, 17, 20, 23, 27], [2, 8, 13, 14, 17, 20, 23, 27])
Max clique: 143679750 143679750
Max clique: ([1, 2, 8, 13, 14, 20, 23, 27], [1, 2, 8, 13, 14, 20, 23, 27])
Max clique: 140613128 140613128
Max clique: ([3, 9, 10, 12, 15, 16, 21, 22, 27], [3, 9, 10, 12, 15, 16, 21, 22, 27])
Max clique: 140612610 140612610
Max clique: ([1, 10, 12, 15, 16, 21, 22, 27], [1, 10, 12, 15, 16, 21, 22, 27])
Max clique: 140612100 140612100
Max clique: ([2, 9, 12, 15, 16, 21, 22, 27], [2, 9, 12, 15, 16, 21, 22, 27])
Max clique: 140611590 140611590
Max clique: ([1, 2, 12, 15, 16, 21, 22, 27], [1, 2, 12, 15, 16, 21, 22, 27])
Max clique: 140580480 140580480
Max clique: ([7, 9, 10, 12, 16, 21, 22, 27], [7, 9, 10, 12, 16, 21, 22, 27])
Max clique: 140579970 140579970
Max clique: ([1, 7, 10, 12, 16, 21, 22, 27], [1, 7, 10, 12, 16, 21, 22, 27])
Max clique: 140579460 140579460
Max clique: ([2, 7, 9, 12, 16, 21, 22, 27], [2, 7, 9, 12, 16, 21, 22, 27])
Max clique: 140578950 140578950
Max clique: (

In [3]:
%%cython --annotate

import myglobals

from importlib import reload 
from math import sqrt,log

reload(myglobals)
#myglobals.bCxt=cxtFromFile("MA7.cxt")




def lst2int(lst):
    r=0
    for i in lst:
        r |= (1<<i)
    return r 

def lst2int_(lst,cxt):
    r=0
    for i in lst:
        r |= (1<<(cxt.nObj-i-1))
    return r 

def int2lst(n):
    ret=[]
    sft=0
    if n==0:
        return ret
    while n!=0:
        if n&1==1:
            ret.append(sft)
        n=n>>1
        sft+=1            
    return ret

def renameList(lst,cxt):
    ret=[]
    for i in lst[::-1]:
        ret.append(cxt.nObj-i-1)
    return ret


def int2lst_(n,cxt):
    ret=[]
    sft=0
    if n==0:
        return ret
    while n!=0:
        if n&1==1:
            ret.append(cxt.nObj-sft-1)
        n=n>>1
        sft+=1            
    return ret
    

class nnBitContext:
    def __init__(self, n=3, i=0, m=3):
            self.attr_names=[str(m+1) for m in range(n)]
            self.objects=[]
            #print(bin(i))
            for j in range(m):                
                self.objects.append(i & ((1<<n) - 1))
                #print(bin(i & ((1<<n) - 1)))
                i=i>>n
                #print(bin(i))
            self.nAttr=len(self.attr_names)
            self.nObj=len(self.objects)


      
 
    def aClosure(self,aSet):
        #вычисление признакового замыкания
        B=(1<<self.nAttr)-1
        for g in self.objects:
            if aSet & g == aSet:
                B=B & g        
        return B
   
    def oClosure(self,oSet):
        #вычисление объектного замыкания
        B=self.oPrime(oSet)
        A=[]
        for i in range(self.nObj):
            if B & self.objects[i] == B:
                A.append(i) 
        #print("A",A)
        return A

    def oPrime(self,oSet):
        #object prime takes a list of objects (indicies) and results in an integer (bitset)
        B=(1<<self.nAttr)-1
        for i in oSet:
            B=B&self.objects[i]
        return B
    
    def aPrime(self,aSet):
        #attribute prime takes an integer (bitset of attributes)
        #and results in a list of objects (the extent)
        A=[]
        for i in range(self.nObj):
            if aSet & self.objects[i]==aSet: 
                A.append(i)
        return A
    
def cxtFromFile(filename):
    cxt=nnBitContext(0,0,0)
   
    f = open(filename, "r")
    print(f.readline())
    print(f.readline())
    
    cxt.nAttr=int(f.readline())
    cxt.nObj=int(f.readline())
    
    print(f.readline())
    
    for i in range(cxt.nObj):
        print(f.readline())
    
    for i in range(cxt.nAttr):
        cxt.attr_names.append(f.readline())
    

    for l in range(cxt.nObj):
        o=f.readline().replace('.','0').replace('x','1')
        print(o)
        
        cxt.objects.append(int(o,2))
        #print(bin(i & ((1<<n) - 1)))
        
        #print(bin(i))
    #print(cxt.objects)
    #cxt.objects.reverse()
    #print("---reverse---")
    #print(cxt.objects)
    return cxt         




def nu(n):
    ret=0
    if n==0:
        return 0
    while n!=0:
        if n&1==1:
            ret+=1
        n=n>>1
    return ret




def CbOFiller(A,g,C,D):
    cnt=0
    parList=[]
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
#         cnt+=1
#         print("Filler:",(C,int2lst(D)))
#         print("Filler:",(renameList(C,myglobals.bCxt),int2lst(D)))
#         print("Filler:",lst2int(C),D)
#         print("Filler:",lst2int_(C,myglobals.bCxt),D)

        intC=lst2int_(C,myglobals.bCxt)

        if intC==D:
            print("Max clique:",intC,D)
            print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))
           
            cnt+=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)
                #print((X,int2lst(Y)))

                parList.append([Z,f,X,Y])

    return cnt,parList


    
    

def processCbO(A,g,C,D):
    
    
 
  
    cnt=0
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
        #cnt+=1
        #print("\t",(C,int2lst(D)))
        #print("\t",(renameList(C,myglobals.bCxt),int2lst(D)))
        #print("\t",lst2int(C),D)
        #print("\t",lst2int_(C,myglobals.bCxt),D)
        intC=lst2int_(C,myglobals.bCxt)

        
        if intC==D:
            #print("Max clique:",intC,D)
            #print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))
            cnt+=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)


                cnt+=processCbO(Z,f,X,Y)
    
    if cnt>0 and cnt%10000==0: print(g,cnt)
    
    return cnt

    

def processCbOWrap(A,g,C,D):
    
    
 
  
    cnt=0
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
        #cnt+=1
        #print("\t",(C,int2lst(D)))
        #print("\t",(renameList(C,myglobals.bCxt),int2lst(D)))
        #print("\t",lst2int(C),D)
        #print("\t",lst2int_(C,myglobals.bCxt),D)
        intC=lst2int_(C,myglobals.bCxt)

        
        if intC==D:
            #print("Max clique:",intC,D)
            #print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))
            cnt+=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)


                cnt+=processCbO(Z,f,X,Y)
    
    if cnt>0 and cnt%10000==0: print(g,cnt)
    
    f=open("CbO_CubeInd"+str(int(log(myglobals.bCxt.nObj,2)))+"_"+str(A)+"_"+str(g)+"_"+str(cnt)+"_log.txt","w")
    f.writelines(str(cnt))
    
    if int(sqrt(myglobals.bCxt.nObj))>=5 and cnt>0: print("branch ("+str(A)+","+str(g)+")"+" terminated with cnt="+str(cnt))
    
    return cnt




In [7]:
%%time

myglobals.bCxt=cxtFromFile("CubeInd5.cxt")

from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp


 



if __name__ == "__main__":
    

    #pool = Pool(cpu_count())
    pool = Pool(1)
    

    
    cnt=0
    
    parList=[]
    
    #if myglobals.bCxt.oClosure([])==[]:
    #    cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))


        parList.extend(lparList)
        
        cnt+=lcnt
    
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res8t = pool.starmap_async(processCbO, parList)
    print(len(res8t.get()))
    print("total:",sum(res8t.get())+cnt)
    #from math import sqrt
    #print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'v

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

10010111011111110111111111111111

01101011101111111011111111111111

01101101110111111101111111111111

10011110111011111110111111111111

01111001111101111111011111111111

10110110111110111111101111111111

11010110111111011111110111111111

11101001111111101111111011111111

01111111100101111111111101111111

10111111011010111111111110111111

11011111011011011111111111011111

11101111100111101111111111101111

11110111011110011111111111110111

11111011101101101111111111111011

11111101110101101111111111111101

11111110111010011111111111111110

01111111111111111001011101111111

10111111111111110110101110111111

11011111111111110110110111011111

11101111111111111001111011101111

11110111111111110111100111110111

11111011111111111011011011111011

11111101111111111101011011111101

11111110111111111110100111111110

11111111011111110111111110010111

1111111110111111101111110

496
total: 1670
CPU times: user 34.3 ms, sys: 16.4 ms, total: 50.6 ms
Wall time: 1.04 s


In [8]:
%%time



from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp


reload(myglobals)

myglobals.bCxt=cxtFromFile("CubeInd5.cxt") 



if __name__ == "__main__":
    

    pool = Pool(10*cpu_count())
    #pool = Pool(1)
    

    
    cnt=0
    
    parList=[]
    
    #if myglobals.bCxt.oClosure([])==[]:
    #    cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))


        parList.extend(lparList)
        
        cnt+=lcnt
    
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res8t = pool.starmap_async(processCbOWrap, parList)
    print(len(res8t.get()))
    print("total:",sum(res8t.get())+cnt)
    #from math import sqrt
    #print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

10010111011111110111111111111111

01101011101111111011111111111111

01101101110111111101111111111111

10011110111011111110111111111111

01111001111101111111011111111111

10110110111110111111101111111111

11010110111111011111110111111111

11101001111111101111111011111111

01111111100101111111111101111111

10111111011010111111111110111111

11011111011011011111111111011111

11101111100111101111111111101111

11110111011110011111111111110111

11111011101101101111111111111011

11111101110101101111111111111101

11111110111010011111111111111110

01111111111111111001011101111111

10111111111111110110101110111111

11011111111111110110110111011111

11101111111111111001111011101111

11110111111111110111100111110111

11111011111111111011011011111011

11111101111111111101011011111101

11111110111111111110100111111110

11111111011111110111111110010111

1111111110111111101111110














branch ([1, 14],14) terminated with cnt=1branch ([0, 12],12) terminated with cnt=1
branch ([8, 11],11) terminated with cnt=1branch ([7, 8],8) terminated with cnt=8branch ([2, 13],13) terminated with cnt=2



branch ([3, 10],10) terminated with cnt=2branch ([2, 11],11) terminated with cnt=2

branch ([9, 10],10) terminated with cnt=1branch ([0, 10],10) terminated with cnt=2

branch ([5, 9],9) terminated with cnt=4
branch ([1, 8],8) terminated with cnt=4branch ([1, 2],2) terminated with cnt=138
branch ([5, 10],10) terminated with cnt=4
branch ([2, 8],8) terminated with cnt=4

branch ([5, 8],8) terminated with cnt=8branch ([4, 9],9) terminated with cnt=8
branch ([6, 8],8) terminated with cnt=8
branch ([1, 6],6) terminated with cnt=147

branch ([3, 6],6) terminated with cnt=54
branch ([5, 6],6) terminated with cnt=54
branch ([3, 9],9) terminated with cnt=4branch ([4, 7],7) terminated with cnt=54branch ([0, 6],6) terminated with cnt=54

branch ([2, 7],7) terminated with cnt=54br

In [9]:
%%time

from time import sleep 

from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp


reload(myglobals)

myglobals.bCxt=cxtFromFile("CubeInd5.cxt") 



if __name__ == "__main__":
    

    pool = Pool(10*cpu_count())
    #pool = Pool(1)
    

    
    cnt=0
    
    parList=[]
    
    #if myglobals.bCxt.oClosure([])==[]:
    #    cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))


        parList.extend(lparList)
        
        cnt+=lcnt
    
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res8t = pool.starmap_async(processCbOWrap, parList)

    print(len(res8t.get()))
    print("total:",sum(res8t.get())+cnt)
    #from math import sqrt
    #print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

10010111011111110111111111111111

01101011101111111011111111111111

01101101110111111101111111111111

10011110111011111110111111111111

01111001111101111111011111111111

10110110111110111111101111111111

11010110111111011111110111111111

11101001111111101111111011111111

01111111100101111111111101111111

10111111011010111111111110111111

11011111011011011111111111011111

11101111100111101111111111101111

11110111011110011111111111110111

11111011101101101111111111111011

11111101110101101111111111111101

11111110111010011111111111111110

01111111111111111001011101111111

10111111111111110110101110111111

11011111111111110110110111011111

11101111111111111001111011101111

11110111111111110111100111110111

11111011111111111011011011111011

11111101111111111101011011111101

11111110111111111110100111111110

11111111011111110111111110010111

1111111110111111101111110

branch ([1, 7],7) terminated with cnt=54branch ([0, 9],9) terminated with cnt=4branch ([1, 10],10) terminated with cnt=8branch ([1, 13],13) terminated with cnt=1branch ([0, 11],11) terminated with cnt=8branch ([2, 11],11) terminated with cnt=2branch ([3, 10],10) terminated with cnt=2branch ([1, 11],11) terminated with cnt=2branch ([0, 15],15) terminated with cnt=1branch ([2, 13],13) terminated with cnt=2branch ([0, 5],5) terminated with cnt=88branch ([3, 12],12) terminated with cnt=2branch ([8, 11],11) terminated with cnt=1branch ([7, 8],8) terminated with cnt=8branch ([9, 10],10) terminated with cnt=1branch ([4, 11],11) terminated with cnt=4










branch ([1, 14],14) terminated with cnt=1
branch ([0, 12],12) terminated with cnt=1

branch ([2, 8],8) terminated with cnt=4branch ([5, 9],9) terminated with cnt=4

branch ([3, 8],8) terminated with cnt=8

branch ([6, 8],8) terminated with cnt=8
branch ([5, 10],10) terminated with cnt=4
branch ([0, 10],10) terminated with cnt=2
branch ([

In [10]:
%%time

from time import sleep 

from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp


reload(myglobals)

myglobals.bCxt=cxtFromFile("CubeInd6.cxt") 



if __name__ == "__main__":
    

    pool = Pool(10*cpu_count())
    #pool = Pool(1)
    

    
    cnt=0
    
    parList=[]
    
    #if myglobals.bCxt.oClosure([])==[]:
    #    cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))


        parList.extend(lparList)
        
        cnt+=lcnt
    
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res8t = pool.starmap_async(processCbOWrap, parList)

    print(len(res8t.get()))
    print("total:",sum(res8t.get())+cnt)
    #from math import sqrt
    #print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

32

33

34

35

36

37

38

39

40

41

42

43

44

45

46

47

48

49

50

51

52

53

54

55

56

57

58

59

60

61

62

63

1001011101111111011111111111111101111111111111111111111111111111

0110101110111111101111111111111110111111111111111111111111111111

0110110111011111110111111111111111011111111111111111111111111111

1001111011101111111011111111111111101111111111111111111111111111

0111100111110111111101111111111111110111111111111111111111111111

1011011011111011111110111111111111111011111111111111111111111111

1101011011111101111111011111111111111101111111111111111111111111

1110100111111110111111101111111111111110111111111111111111111111

0111111110010111111111110111111111111111011111111111111111111111

1011111101101011111111111011111111111111101111111111111111111111

1101111101101101111111111101111111111111110111111111111111111111

111011111001111011111

branch ([4, 19],19) terminated with cnt=8
branch ([14, 16],16) terminated with cnt=16
branch ([16, 19],19) terminated with cnt=1
branch ([17, 18],18) terminated with cnt=1
branch ([1, 19],19) terminated with cnt=4
branch ([2, 19],19) terminated with cnt=4
branch ([3, 18],18) terminated with cnt=4
branch ([0, 23],23) terminated with cnt=2
branch ([1, 14],14) terminated with cnt=774
branch ([3, 20],20) terminated with cnt=4
branch ([0, 17],17) terminated with cnt=8
branch ([5, 16],16) terminated with cnt=16
branch ([10, 16],16) terminated with cnt=16
branch ([8, 19],19) terminated with cnt=8
branch ([15, 16],16) terminated with cnt=16
branch ([2, 21],21) terminated with cnt=4
branch ([2, 13],13) terminated with cnt=1677
branch ([1, 21],21) terminated with cnt=2
branch ([10, 17],17) terminated with cnt=16
branch ([15, 17],17) terminated with cnt=16
branch ([14, 17],17) terminated with cnt=16
branch ([0, 18],18) terminated with cnt=4
branch ([1, 22],22) terminated with cnt=2
branch ([5, 17

In [16]:
sorted([i for i in res8t.get() if i>0])

[1,
 1,
 2,
 2,
 2,
 2,
 4,
 4,
 4,
 4,
 4,
 4,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 128,
 144,
 144,
 144,
 144,
 144,
 144,
 144,
 144,
 144,
 144,
 144,
 144,
 440,
 440,
 440,
 440,
 440,
 440,
 440,
 440,
 774,
 774,
 850,
 850,
 850,
 850,
 1031,
 1031,
 1556,
 1556,
 1556,
 1556,
 1677,
 1677,
 1714,
 1714,
 2594,
 2594,
 3210,
 3210,
 3210,
 3210,
 3741,
 3741,
 6027,
 6027,
 6027,
 6027,
 6027,
 6027,
 8054,
 8054,
 8227,
 8227,
 8227,
 8227,
 8227,
 8227,
 8227,
 8227,
 41859,
 41859,
 41859,
 41859,
 41859,
 41859,
 68682,
 68682,
 68682,
 68682,
 91645,
 91645,
 91645,
 91645,
 109015,
 109015]

In [4]:
%%cython --annotate

import myglobals

from importlib import reload 
from math import sqrt,log

reload(myglobals)
myglobals.dic={}
#myglobals.bCxt=cxtFromFile("MA7.cxt")




def lst2int(lst):
    r=0
    for i in lst:
        r |= (1<<i)
    return r 

def lst2int_(lst,cxt):
    r=0
    for i in lst:
        r |= (1<<(cxt.nObj-i-1))
    return r 

def int2lst(n):
    ret=[]
    sft=0
    if n==0:
        return ret
    while n!=0:
        if n&1==1:
            ret.append(sft)
        n=n>>1
        sft+=1            
    return ret

def renameList(lst,cxt):
    ret=[]
    for i in lst[::-1]:
        ret.append(cxt.nObj-i-1)
    return ret


def int2lst_(n,cxt):
    ret=[]
    sft=0
    if n==0:
        return ret
    while n!=0:
        if n&1==1:
            ret.append(cxt.nObj-sft-1)
        n=n>>1
        sft+=1            
    return ret
    

class nnBitContext:
    def __init__(self, n=3, i=0, m=3):
            self.attr_names=[str(m+1) for m in range(n)]
            self.objects=[]
            #print(bin(i))
            for j in range(m):                
                self.objects.append(i & ((1<<n) - 1))
                #print(bin(i & ((1<<n) - 1)))
                i=i>>n
                #print(bin(i))
            self.nAttr=len(self.attr_names)
            self.nObj=len(self.objects)


      
 
    def aClosure(self,aSet):
        #вычисление признакового замыкания
        B=(1<<self.nAttr)-1
        for g in self.objects:
            if aSet & g == aSet:
                B=B & g        
        return B
   
    def oClosure(self,oSet):
        #вычисление объектного замыкания
        B=self.oPrime(oSet)
        A=[]
        for i in range(self.nObj):
            if B & self.objects[i] == B:
                A.append(i) 
        #print("A",A)
        return A

    def oPrime(self,oSet):
        #object prime takes a list of objects (indicies) and results in an integer (bitset)
        B=(1<<self.nAttr)-1
        for i in oSet:
            B=B&self.objects[i]
        return B
    
    def aPrime(self,aSet):
        #attribute prime takes an integer (bitset of attributes)
        #and results in a list of objects (the extent)
        A=[]
        for i in range(self.nObj):
            if aSet & self.objects[i]==aSet: 
                A.append(i)
        return A
    
def cxtFromFile(filename):
    cxt=nnBitContext(0,0,0)
   
    f = open(filename, "r")
    print(f.readline())
    print(f.readline())
    
    cxt.nAttr=int(f.readline())
    cxt.nObj=int(f.readline())
    
    print(f.readline())
    
    for i in range(cxt.nObj):
        print(f.readline())
    
    for i in range(cxt.nAttr):
        cxt.attr_names.append(f.readline())
    

    for l in range(cxt.nObj):
        o=f.readline().replace('.','0').replace('x','1')
        print(o)
        
        cxt.objects.append(int(o,2))
        #print(bin(i & ((1<<n) - 1)))
        
        #print(bin(i))
    #print(cxt.objects)
    #cxt.objects.reverse()
    #print("---reverse---")
    #print(cxt.objects)
    return cxt         




def nu(n):
    ret=0
    if n==0:
        return 0
    while n!=0:
        if n&1==1:
            ret+=1
        n=n>>1
    return ret




def CbOFiller(A,g,C,D):
    cnt=0
    parList=[]
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
#         cnt+=1
#         print("Filler:",(C,int2lst(D)))
#         print("Filler:",(renameList(C,myglobals.bCxt),int2lst(D)))
#         print("Filler:",lst2int(C),D)
#         print("Filler:",lst2int_(C,myglobals.bCxt),D)

        intC=lst2int_(C,myglobals.bCxt)

        if intC==D:
            print("Max clique:",intC,D)
            print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))
           
            cnt+=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                if f in myglobals.dic[C[-1]]:
                    continue
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)
                #print((X,int2lst(Y)))

                parList.append([Z,f,X,Y])

    return cnt,parList


    
    

def processCbO(A,g,C,D):
    
    
 
  
    cnt=0
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
        #cnt+=1
        #print("\t",(C,int2lst(D)))
        #print("\t",(renameList(C,myglobals.bCxt),int2lst(D)))
        #print("\t",lst2int(C),D)
        #print("\t",lst2int_(C,myglobals.bCxt),D)
        intC=lst2int_(C,myglobals.bCxt)

        
        if intC==D:
            #print("Max clique:",intC,D)
            #print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))
            cnt+=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                if f in myglobals.dic[C[-1]]:
                    continue
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)


                cnt+=processCbO(Z,f,X,Y)
    
    if cnt>0 and cnt%10000==0: print(g,cnt)
    
    return cnt

    

def processCbOWrap(A,g,C,D):
    
    
 
  
    cnt=0
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
        #cnt+=1
        #print("\t",(C,int2lst(D)))
        #print("\t",(renameList(C,myglobals.bCxt),int2lst(D)))
        #print("\t",lst2int(C),D)
        #print("\t",lst2int_(C,myglobals.bCxt),D)
        intC=lst2int_(C,myglobals.bCxt)

        
        if intC==D:
            #print("Max clique:",intC,D)
            #print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))
            cnt+=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                if f in myglobals.dic[C[-1]]:
                    continue
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)


                cnt+=processCbO(Z,f,X,Y)
    
    if cnt>0 and cnt%10000==0: print(g,cnt)
    
    f=open("CbO_CubeInd"+str(int(log(myglobals.bCxt.nObj,2)))+"_"+str(A)+"_"+str(g)+"_"+str(cnt)+"_log.txt","w")
    f.writelines(str(cnt))
    
    if int(sqrt(myglobals.bCxt.nObj))>=5 and cnt>0: print("branch ("+str(A)+","+str(g)+")"+" terminated with cnt="+str(cnt))
    
    return cnt




In [36]:
%%time

from time import sleep 

from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp


reload(myglobals)

myglobals.bCxt=cxtFromFile("CubeInd6.cxt") 

def Prohibition(cxt):
    dic={}
    for i in range(cxt.nObj):
        for j in range(i+1,cxt.nObj):
            if (i&j==i and nu(i)+1==nu(j)):
                if i in dic.keys():
                    dic[i].append(j)
                else: 
                    dic[i]=[j]
    dic[cxt.nObj-1]=[]
    return dic
        
myglobals.dic=Prohibition(myglobals.bCxt)   



if __name__ == "__main__":
    

    pool = Pool(10*cpu_count())
    #pool = Pool(1)
    

    
    cnt=0
    
    parList=[]
    
    #if myglobals.bCxt.oClosure([])==[]:
    #    cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))


        parList.extend(lparList)
        
        cnt+=lcnt
    
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res8t = pool.starmap_async(processCbOWrap, parList)

    print(len(res8t.get()))
    print("total:",sum(res8t.get())+cnt)
    #from math import sqrt
    #print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

32

33

34

35

36

37

38

39

40

41

42

43

44

45

46

47

48

49

50

51

52

53

54

55

56

57

58

59

60

61

62

63

1001011101111111011111111111111101111111111111111111111111111111

0110101110111111101111111111111110111111111111111111111111111111

0110110111011111110111111111111111011111111111111111111111111111

1001111011101111111011111111111111101111111111111111111111111111

0111100111110111111101111111111111110111111111111111111111111111

1011011011111011111110111111111111111011111111111111111111111111

1101011011111101111111011111111111111101111111111111111111111111

1110100111111110111111101111111111111110111111111111111111111111

0111111110010111111111110111111111111111011111111111111111111111

1011111101101011111111111011111111111111101111111111111111111111

1101111101101101111111111101111111111111110111111111111111111111

111011111001111011111

branch ([2, 21],21) terminated with cnt=4
branch ([1, 21],21) terminated with cnt=2
branch ([9, 18],18) terminated with cnt=8
branch ([0, 23],23) terminated with cnt=2
branch ([1, 22],22) terminated with cnt=2
branch ([12, 17],17) terminated with cnt=16
branch ([14, 16],16) terminated with cnt=16
branch ([10, 17],17) terminated with cnt=16
branch ([11, 16],16) terminated with cnt=16
branch ([17, 18],18) terminated with cnt=1
branch ([16, 19],19) terminated with cnt=1
branch ([0, 18],18) terminated with cnt=4
branch ([3, 16],16) terminated with cnt=16
branch ([2, 16],16) terminated with cnt=8
branch ([1, 16],16) terminated with cnt=8
branch ([8, 16],16) terminated with cnt=8
branch ([5, 16],16) terminated with cnt=16
branch ([4, 19],19) terminated with cnt=8
branch ([15, 16],16) terminated with cnt=16
branch ([3, 12],12) terminated with cnt=1677
branch ([11, 17],17) terminated with cnt=16
branch ([14, 17],17) terminated with cnt=16
branch ([0, 19],19) terminated with cnt=16
branch ([15,

In [30]:
dic

{0: [1, 2, 4], 1: [3, 5], 2: [3, 6], 3: [7], 4: [5, 6], 5: [7], 6: [7]}

In [20]:
bin(5)

'0b101'

In [21]:
bin(6)

'0b110'

In [22]:
nu(5)

2

In [23]:
nu(6)

2

In [ ]:
%%time

from time import sleep 

from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp


reload(myglobals)

myglobals.bCxt=cxtFromFile("CubeInd7.cxt") 

def Prohibition(cxt):
    dic={}
    for i in range(cxt.nObj):
        for j in range(i+1,cxt.nObj):
            if (i&j==i and nu(i)+1==nu(j)):
                if i in dic.keys():
                    dic[i].append(j)
                else: 
                    dic[i]=[j]
    dic[cxt.nObj-1]=[]
    return dic
        
myglobals.dic=Prohibition(myglobals.bCxt)   



if __name__ == "__main__":
    

    pool = Pool(10*cpu_count())
    #pool = Pool(1)
    

    
    cnt=0
    
    parList=[]
    
    #if myglobals.bCxt.oClosure([])==[]:
    #    cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))


        parList.extend(lparList)
        
        cnt+=lcnt
    
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res7 = pool.starmap_async(processCbOWrap, parList)

    print(len(res7.get()))
    print("total:",sum(res7.get())+cnt)
    #from math import sqrt
    #print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

32

33

34

35

36

37

38

39

40

41

42

43

44

45

46

47

48

49

50

51

52

53

54

55

56

57

58

59

60

61

62

63

64

65

66

67

68

69

70

71

72

73

74

75

76

77

78

79

80

81

82

83

84

85

86

87

88

89

90

91

92

93

94

95

96

97

98

99

100

101

102

103

104

105

106

107

108

109

110

111

112

113

114

115

116

117

118

119

120

121

122

123

124

125

126

127

10010111011111110111111111111111011111111111111111111111111111110111111111111111111111111111111111111111111111111111111111111111

01101011101111111011111111111111101111111111111111111111111111111011111111111111111111111111111111111111111111111111111111111111

01101101110111111101111111111111110111111111111111111111111111111101111111111111111111111111111111111111111111111111111111111111

1001111011101111111011111111111111101111111111111111111111111111111011111

[[[0, 3], 3, [0, 3], 199962815904344514364775442788909055999], [[0, 5], 5, [0, 5], 200025366859275996235083728065027637247], [[0, 6], 6, [0, 6], 200035792018431243213468442277714067455], [[0, 7], 7, [0, 7], 172127216686525633371680851118170046463], [[0, 9], 9, [0, 9], 201271594165520937346795396971592941567], [[0, 10], 10, [0, 10], 201281981294577893084747936290503131135], [[0, 11], 11, [0, 11], 179604143177864930776980853279135105023], [[0, 12], 12, [0, 12], 201344290758385225578311910991142584319], [[0, 13], 13, [0, 13], 195679495202641214395922074286744928255], [[0, 14], 14, [0, 14], 198504104852142903506011131685231394815], [[0, 15], 15, [0, 15], 199931986567469352543941251873462288383], [[0, 17], 17, [0, 17], 201375034454576629886975904592804446207], [[0, 18], 18, [0, 18], 201375077554697188762503042493732356095], [[0, 19], 19, [0, 19], 180105828579430522100171782699607916543], [[0, 20], 20, [0, 20], 201375322845088446261687847365566791679], [[0, 21], 21, [0, 21], 1960570522574180

In [2]:
%load_ext Cython

In [3]:
%%cython --annotate

import myglobals

from importlib import reload 
from math import sqrt,log

reload(myglobals)
myglobals.dic={}
#myglobals.bCxt=cxtFromFile("MA7.cxt")




def lst2int(lst):
    r=0
    for i in lst:
        r |= (1<<i)
    return r 

def lst2int_(lst,cxt):
    r=0
    for i in lst:
        r |= (1<<(cxt.nObj-i-1))
    return r 

def int2lst(n):
    ret=[]
    sft=0
    if n==0:
        return ret
    while n!=0:
        if n&1==1:
            ret.append(sft)
        n=n>>1
        sft+=1            
    return ret

def renameList(lst,cxt):
    ret=[]
    for i in lst[::-1]:
        ret.append(cxt.nObj-i-1)
    return ret


def int2lst_(n,cxt):
    ret=[]
    sft=0
    if n==0:
        return ret
    while n!=0:
        if n&1==1:
            ret.append(cxt.nObj-sft-1)
        n=n>>1
        sft+=1            
    return ret
    

class nnBitContext:
    def __init__(self, n=3, i=0, m=3):
            self.attr_names=[str(m+1) for m in range(n)]
            self.objects=[]
            #print(bin(i))
            for j in range(m):                
                self.objects.append(i & ((1<<n) - 1))
                #print(bin(i & ((1<<n) - 1)))
                i=i>>n
                #print(bin(i))
            self.nAttr=len(self.attr_names)
            self.nObj=len(self.objects)


      
 
    def aClosure(self,aSet):
        #вычисление признакового замыкания
        B=(1<<self.nAttr)-1
        for g in self.objects:
            if aSet & g == aSet:
                B=B & g        
        return B
   
    def oClosure(self,oSet):
        #вычисление объектного замыкания
        B=self.oPrime(oSet)
        A=[]
        for i in range(self.nObj):
            if B & self.objects[i] == B:
                A.append(i) 
        #print("A",A)
        return A

    def oPrime(self,oSet):
        #object prime takes a list of objects (indicies) and results in an integer (bitset)
        B=(1<<self.nAttr)-1
        for i in oSet:
            B=B&self.objects[i]
        return B
    
    def aPrime(self,aSet):
        #attribute prime takes an integer (bitset of attributes)
        #and results in a list of objects (the extent)
        A=[]
        for i in range(self.nObj):
            if aSet & self.objects[i]==aSet: 
                A.append(i)
        return A
    
def cxtFromFile(filename):
    cxt=nnBitContext(0,0,0)
   
    f = open(filename, "r")
    print(f.readline())
    print(f.readline())
    
    cxt.nAttr=int(f.readline())
    cxt.nObj=int(f.readline())
    
    print(f.readline())
    
    for i in range(cxt.nObj):
        print(f.readline())
    
    for i in range(cxt.nAttr):
        cxt.attr_names.append(f.readline())
    

    for l in range(cxt.nObj):
        o=f.readline().replace('.','0').replace('x','1')
        print(o)
        
        cxt.objects.append(int(o,2))
        #print(bin(i & ((1<<n) - 1)))
        
        #print(bin(i))
    #print(cxt.objects)
    #cxt.objects.reverse()
    #print("---reverse---")
    #print(cxt.objects)
    return cxt         




def nu(n):
    ret=0
    if n==0:
        return 0
    while n!=0:
        if n&1==1:
            ret+=1
        n=n>>1
    return ret




def CbOFiller(A,g,C,D):
    cnt=0
    parList=[]
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
#         cnt+=1
#         print("Filler:",(C,int2lst(D)))
#         print("Filler:",(renameList(C,myglobals.bCxt),int2lst(D)))
#         print("Filler:",lst2int(C),D)
#         print("Filler:",lst2int_(C,myglobals.bCxt),D)

        intC=lst2int_(C,myglobals.bCxt)

        if intC==D:
            print("Max clique:",intC,D)
            print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))
           
            cnt+=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                if f in myglobals.dic[C[-1]]:
                    continue
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)
                #print((X,int2lst(Y)))

                parList.append([Z,f,X,Y])

    return cnt,parList


    
    

def processCbO(A,g,C,D):
    
    
 
  
    cnt=0
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
        #cnt+=1
        #print("\t",(C,int2lst(D)))
        #print("\t",(renameList(C,myglobals.bCxt),int2lst(D)))
        #print("\t",lst2int(C),D)
        #print("\t",lst2int_(C,myglobals.bCxt),D)
        intC=lst2int_(C,myglobals.bCxt)

        
        if intC==D:
            #print("Max clique:",intC,D)
            #print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))
            cnt+=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                mode2=False
                for c in C:
                    if f in myglobals.dic[c]:
                        mode2=True
                        break
                if mode2: continue
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)


                cnt+=processCbO(Z,f,X,Y)
    
    if cnt>0 and cnt%10000==0: print(g,cnt)
    
    return cnt

    

def processCbOWrap(A,g,C,D):
    
    
 
  
    cnt=0
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
        #cnt+=1
        #print("\t",(C,int2lst(D)))
        #print("\t",(renameList(C,myglobals.bCxt),int2lst(D)))
        #print("\t",lst2int(C),D)
        #print("\t",lst2int_(C,myglobals.bCxt),D)
        intC=lst2int_(C,myglobals.bCxt)

        
        if intC==D:
            #print("Max clique:",intC,D)
            #print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))
            cnt+=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                mode2=False
                for c in C:
                    if f in myglobals.dic[c]:
                        #print(f,"in",myglobals.dic[c],"for c=",c)
                        mode2=True
                        break
                if mode2: continue
                #print("Pass")
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)


                cnt+=processCbO(Z,f,X,Y)
    
    if cnt>0 and cnt%10000==0: print(g,cnt)
    
    f=open("CbO_CubeInd"+str(int(log(myglobals.bCxt.nObj,2)))+"_"+str(A)+"_"+str(g)+"_"+str(cnt)+"_log.txt","w")
    f.writelines(str(cnt))
    
    if int(sqrt(myglobals.bCxt.nObj))>=5 and cnt>0: print("branch ("+str(A)+","+str(g)+")"+" terminated with cnt="+str(cnt))
    
    return cnt




In [ ]:
%%time

from time import sleep 

from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp


reload(myglobals)

myglobals.bCxt=cxtFromFile("CubeInd7.cxt") 

def Prohibition(cxt):
    dic={}
    for i in range(cxt.nObj):
        for j in range(i+1,cxt.nObj):
            if (i&j==i and nu(i)+1==nu(j)):
                if i in dic.keys():
                    dic[i].append(j)
                else: 
                    dic[i]=[j]
    dic[cxt.nObj-1]=[]
    return dic

def Cutter(lst,t):
    ret=[]
    for par in lst:
        if par[0][0]<=t and par[0][1]<=t:
            ret.append(par)
    return ret
        
        
        
        
myglobals.dic=Prohibition(myglobals.bCxt)   



if __name__ == "__main__":
    

    pool = Pool(cpu_count())
    #pool = Pool(1)
    

    
    cnt=0
    
    parList=[]
    
    #if myglobals.bCxt.oClosure([])==[]:
    #    cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))
        
        lparList=Cutter(lparList,myglobals.bCxt.nObj/2)


        parList.extend(lparList)
        
        cnt+=lcnt
    
    parList.reverse()
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res7 = pool.starmap_async(processCbOWrap, parList)

    print(len(res7.get()))
    print("total:",sum(res7.get())+cnt)
    #from math import sqrt
    #print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

32

33

34

35

36

37

38

39

40

41

42

43

44

45

46

47

48

49

50

51

52

53

54

55

56

57

58

59

60

61

62

63

64

65

66

67

68

69

70

71

72

73

74

75

76

77

78

79

80

81

82

83

84

85

86

87

88

89

90

91

92

93

94

95

96

97

98

99

100

101

102

103

104

105

106

107

108

109

110

111

112

113

114

115

116

117

118

119

120

121

122

123

124

125

126

127

10010111011111110111111111111111011111111111111111111111111111110111111111111111111111111111111111111111111111111111111111111111

01101011101111111011111111111111101111111111111111111111111111111011111111111111111111111111111111111111111111111111111111111111

01101101110111111101111111111111110111111111111111111111111111111101111111111111111111111111111111111111111111111111111111111111

1001111011101111111011111111111111101111111111111111111111111111111011111

[[[63, 64], 64, [63, 64], 170141183381239855155878339685310791678], [[62, 64], 64, [62, 64], 170141183302010478642939767192465440765], [[61, 64], 64, [61, 64], 170141183143551725727743086649032048635], [[61, 62], 62, [61, 62], 340282366445562204551763869138082070521], [[60, 64], 64, [60, 64], 170141182826634219952689957783293919223], [[60, 63], 63, [60, 63], 340282366207874075234309080544060637174], [[59, 64], 64, [59, 64], 170141182192799211335616007771636367343], [[59, 62], 62, [59, 62], 340282365494809690602358648029715628013], [[59, 61], 61, [59, 61], 340282365336350938277457777844987887595], [[59, 60], 60, [59, 60], 340282365019433431303366284188128903143], [[58, 64], 64, [58, 64], 170141180925129191832518586682046414815], [[58, 63], 63, [58, 63], 340282364306369047335498638327327752158], [[58, 61], 61, [58, 61], 340282364068680917575321991964277080027], [[58, 60], 60, [58, 60], 340282363751763414198345592680780660695], [[57, 64], 64, [57, 64], 170141178389789154597211175578983464

# Part 2. Maximal independence polynomials for n up to 6

In [3]:
%%cython --annotate

import myglobals

from importlib import reload 
from math import sqrt,log

reload(myglobals)
myglobals.dic={}
#myglobals.bCxt=cxtFromFile("MA7.cxt")




def lst2int(lst):
    r=0
    for i in lst:
        r |= (1<<i)
    return r 

def lst2int_(lst,cxt):
    r=0
    for i in lst:
        r |= (1<<(cxt.nObj-i-1))
    return r 

def int2lst(n):
    ret=[]
    sft=0
    if n==0:
        return ret
    while n!=0:
        if n&1==1:
            ret.append(sft)
        n=n>>1
        sft+=1            
    return ret

def renameList(lst,cxt):
    ret=[]
    for i in lst[::-1]:
        ret.append(cxt.nObj-i-1)
    return ret


def int2lst_(n,cxt):
    ret=[]
    sft=0
    if n==0:
        return ret
    while n!=0:
        if n&1==1:
            ret.append(cxt.nObj-sft-1)
        n=n>>1
        sft+=1            
    return ret
    

class nnBitContext:
    def __init__(self, n=3, i=0, m=3):
            self.attr_names=[str(m+1) for m in range(n)]
            self.objects=[]
            #print(bin(i))
            for j in range(m):                
                self.objects.append(i & ((1<<n) - 1))
                #print(bin(i & ((1<<n) - 1)))
                i=i>>n
                #print(bin(i))
            self.nAttr=len(self.attr_names)
            self.nObj=len(self.objects)


      
 
    def aClosure(self,aSet):
        #вычисление признакового замыкания
        B=(1<<self.nAttr)-1
        for g in self.objects:
            if aSet & g == aSet:
                B=B & g        
        return B
   
    def oClosure(self,oSet):
        #вычисление объектного замыкания
        B=self.oPrime(oSet)
        A=[]
        for i in range(self.nObj):
            if B & self.objects[i] == B:
                A.append(i) 
        #print("A",A)
        return A

    def oPrime(self,oSet):
        #object prime takes a list of objects (indicies) and results in an integer (bitset)
        B=(1<<self.nAttr)-1
        for i in oSet:
            B=B&self.objects[i]
        return B
    
    def aPrime(self,aSet):
        #attribute prime takes an integer (bitset of attributes)
        #and results in a list of objects (the extent)
        A=[]
        for i in range(self.nObj):
            if aSet & self.objects[i]==aSet: 
                A.append(i)
        return A
    
def cxtFromFile(filename):
    cxt=nnBitContext(0,0,0)
   
    f = open(filename, "r")
    print(f.readline())
    print(f.readline())
    
    cxt.nAttr=int(f.readline())
    cxt.nObj=int(f.readline())
    
    print(f.readline())
    
    for i in range(cxt.nObj):
        print(f.readline())
    
    for i in range(cxt.nAttr):
        cxt.attr_names.append(f.readline())
    

    for l in range(cxt.nObj):
        o=f.readline().replace('.','0').replace('x','1')
        print(o)
        
        cxt.objects.append(int(o,2))
        #print(bin(i & ((1<<n) - 1)))
        
        #print(bin(i))
    #print(cxt.objects)
    #cxt.objects.reverse()
    #print("---reverse---")
    #print(cxt.objects)
    return cxt         




def nu(n):
    ret=0
    if n==0:
        return 0
    while n!=0:
        if n&1==1:
            ret+=1
        n=n>>1
    return ret




def CbOFiller(A,g,C,D):
    cnt=0
    cntDic={}
    parList=[]
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
#         cnt+=1
#         print("Filler:",(C,int2lst(D)))
#         print("Filler:",(renameList(C,myglobals.bCxt),int2lst(D)))
#         print("Filler:",lst2int(C),D)
#         print("Filler:",lst2int_(C,myglobals.bCxt),D)

        intC=lst2int_(C,myglobals.bCxt)

        if intC==D:
            print("Max clique:",intC,D)
            print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))
           
            if nu(D) in cntDic:
                cntDic[nu(D)]=cntDic[nu(D)]+1
            else: 
                cntDic[nu(D)]=1
            cnt=cnt+1
            
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                if f in myglobals.dic[C[-1]]:
                    continue
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)
                #print((X,int2lst(Y)))

                parList.append([Z,f,X,Y])

    return cnt,cntDic,parList


   
def updateDic(dict1, dict2):    
    for key in dict2:
        if key in dict1:
            dict1[key] = dict2[key] + dict1[key]
        else: 
            dict1[key]=dict2[key]
    return dict1
    

def processCbO(A,g,C,D):
    
    
 
  
    cnt=0
    cntDic={}
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
        #cnt+=1
        #print("\t",(C,int2lst(D)))
        #print("\t",(renameList(C,myglobals.bCxt),int2lst(D)))
        #print("\t",lst2int(C),D)
        #print("\t",lst2int_(C,myglobals.bCxt),D)
        intC=lst2int_(C,myglobals.bCxt)

        
        if intC==D:
            #print("Max clique:",intC,D)
            #print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))
            cnt+=1
            if nu(D) in cntDic:
                cntDic[nu(D)]=cntDic[nu(D)]+1
            else: 
                cntDic[nu(D)]=1
            
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                mode2=False
                for c in C:
                    if f in myglobals.dic[c]:
                        mode2=True
                        break
                if mode2: continue
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)


                cntDic=updateDic(cntDic,processCbO(Z,f,X,Y)[1])
                
                
                
                cnt+=processCbO(Z,f,X,Y)[0]
    
    if cnt>0 and cnt%10000==0: print(g,cnt)
    
    return cnt,cntDic

    

def processCbOWrap(A,g,C,D):
    
    
 
  
    cnt=0
    cntDic={}
    mode=True
    for h in C:
        if (h not in A) and h<g:

            mode=False
            break
    if mode:
        #cnt+=1
        #print("\t",(C,int2lst(D)))
        #print("\t",(renameList(C,myglobals.bCxt),int2lst(D)))
        #print("\t",lst2int(C),D)
        #print("\t",lst2int_(C,myglobals.bCxt),D)
        intC=lst2int_(C,myglobals.bCxt)

        
        if intC==D:
            #print("Max clique:",intC,D)
            #print("Max clique:",(renameList(C,myglobals.bCxt),int2lst(D)))
            cnt+=1
            if nu(D) in cntDic:
                cntDic[nu(D)]=cntDic[nu(D)]+1
            else: 
                cntDic[nu(D)]=1
        elif intC&D==intC:
            F=[i for i in range(myglobals.bCxt.nObj) if (i not in C) and g<i]
            for f in F:
                mode2=False
                for c in C:
                    if f in myglobals.dic[c]:
                        #print(f,"in",myglobals.dic[c],"for c=",c)
                        mode2=True
                        break
                if mode2: continue
                #print("Pass")
                Z=C[:]
                Z.append(f)
                Y = D & myglobals.bCxt.objects[f]
                X = myglobals.bCxt.aPrime(Y)


                cntDic=updateDic(cntDic,processCbO(Z,f,X,Y)[1])
                
                cnt+=processCbO(Z,f,X,Y)[0]
    
    if cnt>0 and cnt%10000==0: print(g,cnt)
    
    f=open("CbO_CubeInd"+str(int(log(myglobals.bCxt.nObj,2)))+"_"+str(A)+"_"+str(g)+"_"+str(cnt)+"_log.txt","w")
    f.writelines(str(cnt))
    
    if int(sqrt(myglobals.bCxt.nObj))>=5 and cnt>0: print("branch ("+str(A)+","+str(g)+")"+" terminated with cnt="+str(cnt))
    
    return cnt,cntDic




In [20]:
%%time

from time import sleep 

from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp


reload(myglobals)

myglobals.bCxt=cxtFromFile("CubeInd2.cxt") 

def Prohibition(cxt):
    dic={}
    for i in range(cxt.nObj):
        for j in range(i+1,cxt.nObj):
            if (i&j==i and nu(i)+1==nu(j)):
                if i in dic.keys():
                    dic[i].append(j)
                else: 
                    dic[i]=[j]
    dic[cxt.nObj-1]=[]
    return dic


def Cutter(lst,t):
    ret=[]
    for par in lst:
        if par[0][0]<=t and par[0][1]<=t:
            ret.append(par)
    return ret
        
        
        
        
myglobals.dic=Prohibition(myglobals.bCxt)   

if __name__ == "__main__":
    

    pool = Pool(cpu_count())
    #pool = Pool(1)
    

    
    cnt=0
    cntDic={}
    
    parList=[]
    
    #if myglobals.bCxt.oClosure([])==[]:
    #    cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lcntDic,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))
        
        #lparList=Cutter(lparList,myglobals.bCxt.nObj)
        

        #parList.extend(lparList)
        
        cnt+=lcnt
        cntDic=updateDic(cntDic,lcntDic)
    
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res2 = pool.starmap_async(processCbOWrap, parList)

    print(len(res2.get()))
    print("total:",sum([p[0] for p in res2.get()])+cnt)
    #from math import sqrt
    #print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

B





0

1

2

3

1001

0110

0110

1001

Max clique: 9 9
Max clique: ([0, 3], [0, 3])
Max clique: 6 6
Max clique: ([1, 2], [1, 2])
[]
0
0
total: 2
CPU times: user 29.5 ms, sys: 55.7 ms, total: 85.2 ms
Wall time: 94.7 ms


[]

In [21]:
from collections import Counter
sum(map(Counter, [p[1] for p in res2.get()]), Counter())

Counter()

In [16]:
%%time

from time import sleep 

from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp


reload(myglobals)

myglobals.bCxt=cxtFromFile("CubeInd3.cxt") 

def Prohibition(cxt):
    dic={}
    for i in range(cxt.nObj):
        for j in range(i+1,cxt.nObj):
            if (i&j==i and nu(i)+1==nu(j)):
                if i in dic.keys():
                    dic[i].append(j)
                else: 
                    dic[i]=[j]
    dic[cxt.nObj-1]=[]
    return dic


def Cutter(lst,t):
    ret=[]
    for par in lst:
        if par[0][0]<=t and par[0][1]<=t:
            ret.append(par)
    return ret
        
        
        
        
myglobals.dic=Prohibition(myglobals.bCxt)   

if __name__ == "__main__":
    

    pool = Pool(cpu_count())
    #pool = Pool(1)
    

    
    cnt=0
    cntDic={}
    
    parList=[]
    
    #if myglobals.bCxt.oClosure([])==[]:
    #    cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lcntDic,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))
        
        lparList=Cutter(lparList,myglobals.bCxt.nObj)
        

        parList.extend(lparList)
        
        cnt+=lcnt
        cntDic=updateDic(cntDic,lcntDic)
    
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res3 = pool.starmap_async(processCbOWrap, parList)

    print(len(res3.get()))
    print("total:",sum([p[0] for p in res3.get()])+cnt)
    #from math import sqrt
    #print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

B





0

1

2

3

4

5

6

7

10010111

01101011

01101101

10011110

01111001

10110110

11010110

11101001

[[[0, 3], 3, [0, 3, 5, 6], 150], [[0, 5], 5, [0, 3, 5, 6], 150], [[0, 6], 6, [0, 3, 5, 6], 150], [[0, 7], 7, [0, 7], 129], [[1, 2], 2, [1, 2, 4, 7], 105], [[1, 4], 4, [1, 2, 4, 7], 105], [[1, 6], 6, [1, 6], 66], [[1, 7], 7, [1, 2, 4, 7], 105], [[2, 4], 4, [1, 2, 4, 7], 105], [[2, 5], 5, [2, 5], 36], [[2, 7], 7, [1, 2, 4, 7], 105], [[3, 4], 4, [3, 4], 24], [[3, 5], 5, [0, 3, 5, 6], 150], [[3, 6], 6, [0, 3, 5, 6], 150], [[4, 7], 7, [1, 2, 4, 7], 105], [[5, 6], 6, [0, 3, 5, 6], 150]]
16
16
total: 6
CPU times: user 28 ms, sys: 44 ms, total: 72 ms
Wall time: 88.1 ms


In [17]:
from collections import Counter
sum(map(Counter, [p[1] for p in res3.get()]), Counter())

Counter({4: 2, 2: 4})

In [12]:
%%time

from time import sleep 

from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp


reload(myglobals)

myglobals.bCxt=cxtFromFile("CubeInd4.cxt") 

def Prohibition(cxt):
    dic={}
    for i in range(cxt.nObj):
        for j in range(i+1,cxt.nObj):
            if (i&j==i and nu(i)+1==nu(j)):
                if i in dic.keys():
                    dic[i].append(j)
                else: 
                    dic[i]=[j]
    dic[cxt.nObj-1]=[]
    return dic


def Cutter(lst,t):
    ret=[]
    for par in lst:
        if par[0][0]<=t and par[0][1]<=t:
            ret.append(par)
    return ret
        
        
        
        
myglobals.dic=Prohibition(myglobals.bCxt)   

if __name__ == "__main__":
    

    pool = Pool(cpu_count())
    #pool = Pool(1)
    

    
    cnt=0
    cntDic={}
    
    parList=[]
    
    #if myglobals.bCxt.oClosure([])==[]:
    #    cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lcntDic,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))
        
        lparList=Cutter(lparList,myglobals.bCxt.nObj/2)
        

        parList.extend(lparList)
        
        cnt+=lcnt
        cntDic=updateDic(cntDic,lcntDic)
    
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res4 = pool.starmap_async(processCbOWrap, parList)

    print(len(res4.get()))
    print("total:",sum([p[0] for p in res4.get()])+cnt)
    #from math import sqrt
    #print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

1001011101111111

0110101110111111

0110110111011111

1001111011101111

0111100111110111

1011011011111011

1101011011111101

1110100111111110

0111111110010111

1011111101101011

1101111101101101

1110111110011110

1111011101111001

1111101110110110

1111110111010110

1111111011101001

[[[0, 3], 3, [0, 3], 38511], [[0, 5], 5, [0, 5], 38523], [[0, 6], 6, [0, 6], 38525], [[0, 7], 7, [0, 7], 33150], [[1, 2], 2, [1, 2], 27039], [[1, 4], 4, [1, 4], 27063], [[1, 6], 6, [1, 6], 17085], [[1, 7], 7, [1, 7], 27070], [[1, 8], 8, [1, 8], 27543], [[2, 4], 4, [2, 4], 27095], [[2, 5], 5, [2, 5], 9435], [[2, 7], 7, [2, 7], 27102], [[2, 8], 8, [2, 8], 28055], [[3, 4], 4, [3, 4], 6375], [[3, 5], 5, [3, 5], 38635], [[3, 6], 6, [3, 6], 38637], [[3, 8], 8, [3, 8], 7815], [[4, 7], 7, [4, 7], 27126], [[4, 8], 8, [4, 8], 31127], [[5, 6], 6, [5, 6], 38649], [[5, 8], 8, [5, 8], 13971], [[6, 8], 8, [6, 8], 22165], [[7, 8], 8, [1, 2, 4, 7, 8, 11, 13, 1

In [13]:
from collections import Counter
sum(map(Counter, [p[1] for p in res4.get()]), Counter())

Counter({8: 2, 5: 16, 4: 24})

In [5]:
%%time

from time import sleep 

from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp


reload(myglobals)

myglobals.bCxt=cxtFromFile("CubeInd5.cxt") 

def Prohibition(cxt):
    dic={}
    for i in range(cxt.nObj):
        for j in range(i+1,cxt.nObj):
            if (i&j==i and nu(i)+1==nu(j)):
                if i in dic.keys():
                    dic[i].append(j)
                else: 
                    dic[i]=[j]
    dic[cxt.nObj-1]=[]
    return dic


def Cutter(lst,t):
    ret=[]
    for par in lst:
        if par[0][0]<=t and par[0][1]<=t:
            ret.append(par)
    return ret
        
        
        
        
myglobals.dic=Prohibition(myglobals.bCxt)   

if __name__ == "__main__":
    

    pool = Pool(cpu_count())
    #pool = Pool(1)
    

    
    cnt=0
    cntDic={}
    
    parList=[]
    
    #if myglobals.bCxt.oClosure([])==[]:
    #    cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lcntDic,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))
        
        lparList=Cutter(lparList,myglobals.bCxt.nObj/2)
        

        parList.extend(lparList)
        
        cnt+=lcnt
        cntDic=updateDic(cntDic,lcntDic)
    
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res5 = pool.starmap_async(processCbOWrap, parList)

    print(len(res5.get()))
    print("total:",sum([p[0] for p in res5.get()])+cnt)
    #from math import sqrt
    #print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

10010111011111110111111111111111

01101011101111111011111111111111

01101101110111111101111111111111

10011110111011111110111111111111

01111001111101111111011111111111

10110110111110111111101111111111

11010110111111011111110111111111

11101001111111101111111011111111

01111111100101111111111101111111

10111111011010111111111110111111

11011111011011011111111111011111

11101111100111101111111111101111

11110111011110011111111111110111

11111011101101101111111111111011

11111101110101101111111111111101

11111110111010011111111111111110

01111111111111111001011101111111

10111111111111110110101110111111

11011111111111110110110111011111

11101111111111111001111011101111

11110111111111110111100111110111

11111011111111111011011011111011

11111101111111111101011011111101

11111110111111111110100111111110

11111111011111110111111110010111

1111111110111111101111110

In [7]:
from collections import Counter
sum(map(Counter, [p[1] for p in res5.get()]), Counter())

Counter({16: 2, 12: 32, 10: 176, 9: 320, 8: 1140})

In [8]:
%%time

from time import sleep 

from multiprocess import Pool
from multiprocess import cpu_count
import multiprocess as mp


reload(myglobals)

myglobals.bCxt=cxtFromFile("CubeInd6.cxt") 

def Prohibition(cxt):
    dic={}
    for i in range(cxt.nObj):
        for j in range(i+1,cxt.nObj):
            if (i&j==i and nu(i)+1==nu(j)):
                if i in dic.keys():
                    dic[i].append(j)
                else: 
                    dic[i]=[j]
    dic[cxt.nObj-1]=[]
    return dic


def Cutter(lst,t):
    ret=[]
    for par in lst:
        if par[0][0]<=t and par[0][1]<=t:
            ret.append(par)
    return ret
        
        
        
        
myglobals.dic=Prohibition(myglobals.bCxt)   

if __name__ == "__main__":
    

    pool = Pool(cpu_count())
    #pool = Pool(1)
    

    
    cnt=0
    cntDic={}
    
    parList=[]
    
    #if myglobals.bCxt.oClosure([])==[]:
    #    cnt+=1
        
    for i in range(myglobals.bCxt.nObj):
        
        lcnt,lcntDic,lparList=CbOFiller([i],i,myglobals.bCxt.oClosure([i]),myglobals.bCxt.oPrime([i]))
        
        lparList=Cutter(lparList,myglobals.bCxt.nObj/2)
        

        parList.extend(lparList)
        
        cnt+=lcnt
        cntDic=updateDic(cntDic,lcntDic)
    
    print(parList)
    print(len(parList))
    #pool = Pool(len(parList))
       
     
    

    
    
    res6 = pool.starmap_async(processCbOWrap, parList)

    print(len(res6.get()))
    print("total:",sum([p[0] for p in res6.get()])+cnt)
    #from math import sqrt
    #print(sqrt(sum(res8t.get())+cnt))
    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

B





0

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22

23

24

25

26

27

28

29

30

31

32

33

34

35

36

37

38

39

40

41

42

43

44

45

46

47

48

49

50

51

52

53

54

55

56

57

58

59

60

61

62

63

1001011101111111011111111111111101111111111111111111111111111111

0110101110111111101111111111111110111111111111111111111111111111

0110110111011111110111111111111111011111111111111111111111111111

1001111011101111111011111111111111101111111111111111111111111111

0111100111110111111101111111111111110111111111111111111111111111

1011011011111011111110111111111111111011111111111111111111111111

1101011011111101111111011111111111111101111111111111111111111111

1110100111111110111111101111111111111110111111111111111111111111

0111111110010111111111110111111111111111011111111111111111111111

1011111101101011111111111011111111111111101111111111111111111111

1101111101101101111111111101111111111111110111111111111111111111

111011111001111011111

branch ([1, 21],21) terminated with cnt=2
branch ([1, 22],22) terminated with cnt=2
branch ([3, 17],17) terminated with cnt=8
branch ([0, 15],15) terminated with cnt=774
branch ([3, 18],18) terminated with cnt=4
branch ([3, 20],20) terminated with cnt=4
branch ([0, 17],17) terminated with cnt=8
branch ([2, 13],13) terminated with cnt=1677
branch ([0, 18],18) terminated with cnt=4
branch ([0, 19],19) terminated with cnt=16
branch ([0, 20],20) terminated with cnt=2
branch ([0, 23],23) terminated with cnt=2
branch ([4, 13],13) terminated with cnt=440
branch ([2, 14],14) terminated with cnt=128
branch ([4, 14],14) terminated with cnt=128
branch ([2, 15],15) terminated with cnt=144
branch ([2, 16],16) terminated with cnt=8
branch ([2, 17],17) terminated with cnt=16
branch ([2, 19],19) terminated with cnt=4
branch ([4, 15],15) terminated with cnt=144
branch ([2, 21],21) terminated with cnt=4
branch ([4, 16],16) terminated with cnt=8
branch ([6, 16],16) terminated with cnt=16
branch ([4, 17],

In [9]:
from collections import Counter
sum(map(Counter, [p[1] for p in res6.get()]), Counter())


Counter({32: 2,
         27: 64,
         24: 480,
         22: 1856,
         20: 8320,
         21: 3840,
         19: 40320,
         18: 116320,
         17: 337920,
         16: 736440,
         15: 25920,
         14: 9600,
         12: 320})

In [10]:
res6.get()

[(109015,
  {32: 1,
   27: 22,
   24: 124,
   22: 384,
   20: 1277,
   21: 696,
   19: 5428,
   18: 13028,
   17: 31432,
   16: 54753,
   15: 1422,
   14: 440,
   12: 8}),
 (68682,
  {27: 3,
   22: 114,
   19: 2386,
   24: 28,
   21: 237,
   17: 18541,
   20: 509,
   18: 6652,
   16: 38670,
   14: 365,
   15: 1170,
   12: 7}),
 (41859,
  {24: 4,
   19: 904,
   21: 66,
   18: 3088,
   20: 174,
   16: 26019,
   22: 24,
   17: 10366,
   15: 918,
   14: 290,
   12: 6}),
 (91645,
  {27: 2,
   24: 24,
   22: 110,
   21: 246,
   20: 610,
   19: 2892,
   18: 8654,
   17: 24546,
   16: 52005,
   15: 1908,
   14: 630,
   12: 18}),
 (6027,
  {22: 2,
   19: 117,
   18: 441,
   17: 1533,
   16: 3720,
   21: 5,
   20: 13,
   15: 136,
   14: 58,
   12: 2}),
 (3210,
  {20: 3, 17: 642, 18: 151, 16: 2195, 19: 34, 15: 124, 14: 58, 12: 2, 21: 1}),
 (8227,
  {19: 167,
   18: 568,
   17: 2054,
   16: 4988,
   15: 273,
   14: 145,
   12: 7,
   22: 3,
   21: 8,
   20: 14}),
 (1031, {19: 10, 17: 186, 18: 43, 1

In [11]:
len(res6.get())

447